In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import re
from collections import defaultdict

In [2]:
with open("varia.xml", "r", encoding="utf-8") as file:
    # Read each line in the file, readlines() returns a list of lines
    content = file.readlines()
    # Combine the lines in the list into a string
    content = "".join(content)
    soup = bs(content, "xml")
with open("navneregister.xml", "r", encoding="utf-8") as file:
    # Read each line in the file, readlines() returns a list of lines
    content = file.readlines()
    # Combine the lines in the list into a string
    content = "".join(content)
    registry = bs(content, "xml")

In [3]:
d = defaultdict(dict)

docDates,regDates,mscontentscount,biblcount = 0,0,0,0
i=0
print("Working...")
for item in soup.findAll("TEI"): # For every TEI element (every document) in the xml file:
    itemDate = "N/A" # I don't think this is required
    itemType = item['subtype'] # Check the subtype of the document
    if itemType == "dedication" or itemType == "greeting": # Include dedications and greetings only
        xmlid = item['xml:id'] # Find the XML ID of the document
        #print(item["xml:id"])
        
        # First off, try to find the title in the bibl element.
        try: # Try clause because title = None will throw an exception
            bibl = item.find("bibl")
            title = bibl.find("title")
            #print("Title in bibl element")
            biblcount += 1
        # If exception, there's no title in bibl. Look for title in msContents:
        except:
            bibl = item.find("msContents")
            title = bibl.find("title")
            #print("Title in msContents element")
            mscontentscount += 1
        # With the title, unwrap any subelements
        for match in title.findAll():
            match.unwrap()
        newTitle = ""
        for x in title.contents: # Split and recombine the title into 1 whole string
            newTitle+=" "+x
        title = re.sub(' +', ' ',title) # Remove double+ spaces
        title = newTitle.strip() # Remove spaces around the title
        title = re.sub('\n',"",title) # Remove \n
        itemDate = item.find("docDate") # Look for the date in docDate element
        try: # If no docDate, will throw exception
            xmldate = itemDate['when']
            docDates+=1

        except: # No docDate? Look for date in date element
            itemDate = item.find("date")
            xmldate = itemDate['when']
            regDates+=1
        date = itemDate['when']
        datenumbers = xmldate.replace("-","") # Get the numbers only from the date (1000-10-10 = 10001010)
        # Because xml:ids are V(udat)1000-10-10ID
        
        if "Vudat" in xmlid: # Prefix for some xml:ids
            persid = xmlid.replace("Vudat","")
        else:
            persid = xmlid[1:] # Prefix for the rest of the xml:ids
        persid = persid.replace(datenumbers,"")
        if "-" in persid: # Indicates date range in xmlid.
            persid = persid[3:] # Remove -XX left behind.
        #print("\n")
        
        # Collect all known information in dict d
        d[xmlid]['title'] = title
        d[xmlid]['type'] = itemType
        d[xmlid]['date'] = date
        d[xmlid]['ID'] = persid
        
        #print(xmlid,title,itemType,date,"pe/orgID:",persid)
        i+=1
print(i,"total","\ndocDate elements:",docDates,"\nregular date elements:",regDates,"\nmscontentscount:",mscontentscount,"\nbibl count:",biblcount)

Working...
179 total 
docDate elements: 159 
regular date elements: 20 
mscontentscount: 159 
bibl count: 20


In [4]:
iP=0 # Count of persons
iO=0 # Count of organizations
found_persons = [] # Applies to both, where we have a title with a positive ID identified
print("Working...")
for item in d: # For every dict entry
    whoid = d[item]['ID'] # Get the ID
    persid = "pe"+whoid # Test for personal ID
    orgid = "org"+whoid # Test for organization ID
    try: # Try clause because attempting to use None below will throw an exception
        persdiv = registry.find('div', {'xml:id':persid}) # Test for personal ID
        name = persdiv.find('persName') # Attempt to retrieve personal name
        for match in name.findAll():
            match.unwrap() # Removes subelements
        newname = ""
        for x in name.contents: # Can't use title.contents[0]. Making new string as workaround.
            newname+=" "+x
        name = newname.strip()
        name = re.sub(' +', ' ',name)
        #print("PERS - Title:",d[item]['title'],"To:",d[item]['ID'],"Registry:",name) # Report
        iP+=1
        found_persons.append(item)
        d[item]['peID'] = "pe"+whoid
        d[item]['clearname'] = name
        # Person confirmed
    except: # If an exception is thrown, it's not a person
        try: # Test for organization ID
            orgid = registry.find('div', {'xml:id':orgid})
            name = orgid.find('item',{'rend':'name'})
            for match in name.findAll():
                match.unwrap()
            newname = ""
            for x in name.contents: # Can't use title.contents[0]. Making new string as workaround.
                newname+=" "+x
            name = newname.strip()
            name = re.sub(' +', ' ',name)
            #print("ORG - Title:",d[item]['title'],"To:",d[item]['ID'],"Registry:",name) # Report
            iO+=1
            found_persons.append(item)
            d[item]['orgID'] = "org"+whoid
            d[item]['clearname'] = name
        except: # If an exception is thrown, it's not an organization
            pass # Report or pass.

print(iP,"personal names\n",iO,"organizations")

Working...
92 personal names
 1 organizations


In [5]:
losttitles = [] # List of items that do not have a person/organization's ID *and* is not attributed to an anon
for item in d: 
    if item not in found_persons and d[item]['ID'] != "NN":
        #print(item,"not found. Title",d[item]['title'],"ID:",d[item]['ID'])
        losttitles.append(d[item]['title'])

In [6]:
# Get full names and their IDs from the registry xml file
i=0
iddict = defaultdict(dict)
orgs = []
pers = []
for orgid in registry.findAll('div', {'xml:id':re.compile('org.*')}):
    name = orgid.find('item',{'rend':'name'})
    for match in name.findAll():
        match.unwrap()
    newname = ""
    for x in name.contents: # Can't use title.contents[0]... New string as workaround.
        newname+=" "+x
    newname = re.sub('\\n',"",newname)
    name = newname.strip()
    name = re.sub(' +', ' ',name)
    print(name)
    i+=1
    orgs.append(name)
    iddict[name]["ID"] = orgid['xml:id']
print(i,"organizations")
i=0
for persid in registry.findAll('div', {'xml:id':re.compile('pe.*')}):
    name = persid.find('item',{'rend':'name'})
    for match in name.findAll():
        match.unwrap()
    newname = ""
    
    for x in name.contents: # Can't use title.contents[0]... New string as workaround.
        newname+=" "+x
    name = newname
    name = re.sub('\\n',"",name)
    name = name.strip()
    name = re.sub(' +', ' ',name)
    print(name)
    i+=1
    iddict[name]["ID"] = persid['xml:id']
    pers.append(name)
print(i,"persons")

GEORG 2.
SILLE (Cecilie Marie) f. Gade JØRGENSEN
AFTENBLADET
AFTENPOSTEN
BANCO NAPOLETANA
BERGENS NATIONALE SCENE
BERGENS THEATERFORENING
BERLINGSKE TIDENDE
BURGTHEATER
CHRISTIANIA BYFOGEDCONTOR
CHRISTIANIA THEATER
CHRISTIANIA-POSTEN
DAGBLADET
DAGS-TELEGRAPHEN
DEN FRISINNEDE STUDENTERFORENING
DEN GYLDENDALSKE BOGHANDEL
DEN NORSKE FORFATTERFORENING
DEN SKANDINAVISKE FORENING I ROMA
DET AKADEMISKE KOLLEGIUM
DET BRITISK-SKANDINAVISKE SELSKAB
DET KONGELIGE NORSKE VIDENSKABERS SELSKAB
DET KONGELIGE THEATER I KØBENHAVN
DET NORSKE STUDENTERSAMFUND
DET NORSKE THEATER I BERGEN
DEUTSCHE GENOSSENSCHAFT DRAMATISCHER AUTOREN UND COMPONISTEN
DEUTSCHES VOLKSTHEATER
DIREKSJONEN FOR DAMPSKIPET DRONNINGEN
DRESDNER BEZIRKS-GERICHTS-AMT
FELIX BLOCH ERBEN
FINANTS- OG TOLDDEPARTEMENTET (FINANSDEPARTEMENTET)
GIORDANO BRUNO-KOMITEEN
HOHLENBERG
HOTEL GRÖBNER
ILLUSTRERET NYHEDSBLAD
JENSEN CO.
KIRKE- OG UNDERVISNINGSDEPARTEMENTET, eller: DEPARTEMENTET FOR KIRKE- OG UNDERVISNINGSVESENET, gjerne forkortet: KIRKEDE

WILHELM BÖLSCHE
JOHN (Johan Laurentius Helenus) BÖRJESON
HERMAN [BRUNCHORST] BAARS
MARGRETHE ELISABETH f. Konow BAARS
GAIUS IULIUS CAESAR
CAESARIUS fra Kappadokia
JACOB [MATHIAS] CALMEYER
[FREDERIK] ALBERT CAMMERMEYER
JØRGEN WRIGHT CAPPELEN
SOFIE [LOUISE] CAPPELEN
LUIGI CAPUANA
ALBERT CARRÉ
[PHILIPP] MORITZ CARRIERE
[HANS] CARL [EMIL] CARSTENSEN
CARL PAUL CASPARI
CATHARINA [MAGDALENA] f. Wahl CASPARI
[CARL] THEODOR CASPARI
RICCARDO (pseudonym for grev Giulio Pullè) CASTELVECCHIO
[PAULUS] HENRIK CAVLING
FREDERIK [ADOLPH CHRISTENSEN] CETTI
CHARLOTTE
CHARLOTTE
CHARLES CHARRINGTON
[VICTOR EUPHÉMION] PHILARÈTE CHASLES
MICHEL CHEVALIER
HJALMAR CHRISTENSEN
CHRISTIAN 4.
CHRISTIAN 7.
CHRISTIAN 9.
[ARNE] EINAR CHRISTIANSEN
EDVARD CHRISTIE
KRISTINE [MARIE] f. Gotaas Kildal CHRISTIE
CHRISTIAN [EILERT RASCH] CHRISTOPHERSEN
WILHELM [CHRISTOPHER] CHRISTOPHERSEN
LUDWIG CHRONEGK
IGNAZIO CIAMPI
MARCUS TULLIUS CICERO
JULES (eg. Arsène Armand) CLARETIE
HENRIK NICOLAI CLAUSEN
CONRAD CLAUSON
LOUISA CLAUSON


PETER FRIEDRICH SIEBOLD
SIGURD 1. Magnusson JORSALFARE
CHRISTINE [MARIE MAGDALENE] SIEWERS
PETER SIMONSEN
ANNA [CHRISTINE] f. Nielsen SINDING
CHRISTIAN [AUGUST] SINDING
OTTO [LUDVIG] SINDING
SIGMUND SINDING
STEPHAN [ABEL] SINDING
[NILS ULRIK] ALFRED SINDING-LARSEN (før 1881: Sinding Larsen)
PER [ERIKSEN] (Peder) SIVLE
EDVARD [OTTO ROSSING] SKARI
OLAF (Ole) SKAVLAN
MATHIAS [SEVERIN BERNTSEN] SKEIBROK
[BERTHE] AMALIE f. Alver SKRAM
CHRISTIAN [ERICHSEN] SKREDSVIG
JOHANNES [NILSSON] SKAAR
CARL FRITHJOF SMITH
ERNEST WILLIAM SMITH
MATHILDE [JOSEPHINE] SMITH
[ISAC] FREDRIK SMITT
CARL [JOHAN GUSTAF] SNOILSKY
EBBA [FREDRIKA ELEONORA] friherrinne f. Ruuth SNOILSKY
ELISABETH [FLORENTINA] SNOILSKY
SNORRE STURLASSON
SOFIE [WILHELMINE MARIANE HENRIETTE] (Sophie; i Sverige Sofia eller Sophia)
[PER] AUGUST [FERDINAND] SOHLMAN
RUDOLF SOKOLOWSKY
JØRGEN [VALENTIN] SONNE
BOLETTE SONTUM
CAROLINE [EUGENIE] f. Næser SONTUM
CHRISTIAN [FREDRIK] SONTUM
CHRISTIAN FREDRIK SONTUM
HELENE [MØLLER] f. Linde SONTUM
SO

In [7]:
# Macerate the list of missing titles in an attempt to extract the name(s) included in each title by preposition guessing.
# This method has a success rate of, like, 75%. It's not perfect, but extracts a reasonable number of names.
# I suspect that chances of success would increase by creating a list of terms that should just be excluded entirely -
# like a stop word list - and analysing more thoroughly where each name is placed in relation to a preposition.
titlesstillmissing,nodontadd,fulltitlestillmissing = [],[],[]
v=0
for x in losttitles:
    z = x.strip()
    #print("Mod 0:",z)
    z = z.replace("Dedikasjon til","")
    z = z.replace("Hilsen til","")
    z = z.replace("Hilsener til","")
    #print("Mod 1:",z)
    if " i " in z:
        a = z.split(" i ")
    elif " på " in z:
        a = z.split(" på ")
    elif " til " in z:
        a = z.split(" til ")
    elif " bakpå " in z:
        a = z.split(" bakpå ")
    else:
        a = [z]
        #print("Neither",x)
    if "Hilsen" in a[0]:
        z = a[1]
    else:
        z = a[0]
    #print(z)
    z = z.strip()
    #print("Mod 2:",z)
    
    # Check the names that we got against the registry - organizations
    for xx in orgs:
        if z.lower() in xx.lower():
            print("Mod 3O:",x,"/",z,"/",xx)
            v+=1
            nodontadd.append(z)
            for document in d:
                #print(x)
                exists = d.get(document, {}).get('title')
                if exists == x:
                    theid = iddict.get(xx)["ID"]
                    d[document]['orgID'] = theid
                    d[document]['clearname'] = xx.strip()
                    print(x,"/",xx.strip(),exists,"updated to ID",theid)
    # Check the names that we got against the registry - persons
    for xx in pers:
        if z.lower() in xx.lower():
            #print("Mod 3P:",x,"/",z,"/",xx)
            v+=1
            nodontadd.append(z)
            for document in d:
                #print(x)
                exists = d.get(document, {}).get('title')
                if exists == x:
                    theid = iddict.get(xx)["ID"]
                    d[document]['peID'] = theid
                    d[document]['clearname'] = xx.strip()
                    print(x,"/",xx.strip(),exists,"updated to ID",theid)
    if z not in nodontadd:
        titlesstillmissing.append(z)
        fulltitlestillmissing.append(x)
    #print("")
print(v,"new entries")

Mod 3O: Dedikasjon til Den skandinaviske Forening i Rom i Nytaarsgave for Illustreret Nyhedsblads Abonnenter av Paul Botten Hansen / Den skandinaviske Forening / DEN SKANDINAVISKE FORENING I ROMA
Dedikasjon til Den skandinaviske Forening i Rom i Nytaarsgave for Illustreret Nyhedsblads Abonnenter av Paul Botten Hansen / DEN SKANDINAVISKE FORENING I ROMA Dedikasjon til Den skandinaviske Forening i Rom i Nytaarsgave for Illustreret Nyhedsblads Abonnenter av Paul Botten Hansen updated to ID orgSF
Dedikasjon til Den skandinaviske Forening i Rom i Nytaarsgave for Illustreret Nyhedsblads Abonnenter av Paul Botten Hansen / DEN SKANDINAVISKE FORENING I ROMA Dedikasjon til Den skandinaviske Forening i Rom i Nytaarsgave for Illustreret Nyhedsblads Abonnenter av Paul Botten Hansen updated to ID orgSF
Dedikasjon til Den skandinaviske Forening i Rom i Nytaarsgave for Illustreret Nyhedsblads Abonnenter av Paul Botten Hansen / DEN SKANDINAVISKE FORENING I ROMA Dedikasjon til Den skandinaviske Forening

In [8]:
# Debug and report step. Creates fullID column in the dictionary.
i=0
for x in d:
    #print(x)
    peID = d.get(x, {}).get('peID')
    orgID = d.get(x, {}).get('orgID')
    ID = d.get(x, {}).get('ID')
    clearname = d.get(x, {}).get('clearname')
    if peID:
        i+=1
        print("PEID",x,"=",ID,"/",peID,"=",clearname)
        print(i)
        d[x]['fullID'] = peID
    elif orgID:
        i+=1
        print("ORGID",x,"=",ID,"/",orgID,"=",clearname)
        print(i)
        d[x]['fullID'] = orgID

PEID V18500415CLD = CLD / peCLD = CHRISTOPHER [LORENTZ] DUE
1
PEID V18571004MB = MB / peMB = MICHAEL BIRKELAND
2
PEID V185712JCG = JCG / peJCG = JOACHIM [CHRISTIAN] GAMBORG
3
PEID V1858CRU = CRU / peCRU = CARL RICHARD UNGER
4
PEID V1858KK = KK / peKK = KITTY (Christine Lange) KIELLAND
5
PEID V1858CHo = CHo / peCHo = CHRISTIAN HOLST
6
PEID V18600819ABla = ABla / peABla = AUGUST [THEODOR] BLANCHE
7
ORGID V1860Skand = Skand / orgSF = DEN SKANDINAVISKE FORENING I ROMA
8
ORGID V1861Skand = Skand / orgSF = DEN SKANDINAVISKE FORENING I ROMA
9
ORGID V1862Skand = Skand / orgSF = DEN SKANDINAVISKE FORENING I ROMA
10
PEID V1862WHo = WHo / peWHo = WILHELMINE HOLMBOE (Wielma Lhombino) SCHENSTRØM
11
PEID V1863BD = BD / peBD = [CARL CHRISTIAN HENRIK] BERNHARD DUNKER
12
PEID V18641012CBl = CBl / peCBl = CARL [HEINRICH] BLOCH
13
PEID V1864MB = MB / peMB = MICHAEL BIRKELAND
14
PEID Vudat1866-70AK = AK / peAK = ANTON [HIERONYMUS FRITZSCHE] KLUBIEN [klubin]
15
PEID V18671221Mo = Mo / peMThr = MARCUS [MØLL

In [9]:
# Token matching by Ruth. This is an additional effort to mash up titles that still haven't received a recipient.
# Uses the partial titles that were macerated based on proposition guessing earlier.

PossibleTokenMatchReg = []
LeftoverNames = []
                      
for entry in titlesstillmissing:
    string = re.sub('\(', '',(re.sub('\)', '', str(entry))))
    stringLower = string.lower()
    tokens = stringLower.split()
    TokenMatchReg = []            
    for person in pers:
        personLower = person.lower()
        personTokens = re.sub('\(', '',(re.sub('\)', '',(re.sub('\]', '',(re.sub('\[', '', personLower)))))))
    # personTokens = personTokens.split()
        
        if all(x in personTokens for x in tokens):
            index = titlesstillmissing.index(entry)
            variaID = ''
            for item in d:
                if d[item]['title'] == fulltitlestillmissing[index]:
                    variaID = item
                
            TokenMatchReg.append(variaID + ':' + fulltitlestillmissing[index] + '/' + person)
    
    for org in orgs:
        orgLower = org.lower()
        orgTokens = re.sub('\(', '',(re.sub('\)', '',(re.sub('\]', '',(re.sub('\[', '', orgLower)))))))
        
        if all(x in orgTokens for x in tokens):
            index = titlesstillmissing.index(entry)
            variaID = ''
            for item in d:
                if d[item]['title'] == fulltitlestillmissing[index]:
                    variaID = item
            TokenMatchReg.append( variaID + ':' + fulltitlestillmissing[index] + '/' + orgTokens)
        
        
    if len(TokenMatchReg) != 0:
        PossibleTokenMatchReg.append(TokenMatchReg)
    else:
            LeftoverNames.append(entry)
        
print(PossibleTokenMatchReg)
# print(LeftoverNames)
# print(len(LeftoverNames))

[['V18690926HSTp:Hilsen til Hedvig Stousland på fotografi/HEDVIG (ved dåpen Hedevig Cathrine) f. Ibsen STOUSLAND'], ['V18880221Aub:Hilsen til Ludvig Aubert på fotografi/[FREDRIK LUDVIG] ANDREAS [VIBE] AUBERT', 'V18880221Aub:Hilsen til Ludvig Aubert på fotografi/LUDVIG [MARIBO BENJAMIN] AUBERT'], ['V18901219HeG:Dedikasjon til Hedwig Grønvold i Hedda Gabler/HEDWIG f. Hermann GRØNVOLD'], ['V1890HJae:Dedikasjon til Henrik Jæger i Hedda Gabler/HANS [HENRIK] JÆGER', 'V1890HJae:Dedikasjon til Henrik Jæger i Hedda Gabler/HENRIK [BERNHARD] JÆGER']]


In [20]:
# Match fullnames to iddict for ID
# Create list for inexact matches
idsfound,ambignames,ambigids,ambig,ambigdocs=[],[],[],[],[]
for x in PossibleTokenMatchReg:
    for y in x:
        z = y.split(":")
        docID = z[0]
        names = z[1].split("/")
        name = names[1]
        name = name.strip()
        if docID in idsfound:
            print("-AMBIGUOUS:",docID,name,iddict[name]['ID'])
            ambig.append(docID)
        else:
            #print(docID,name)
            idsfound.append(docID)
for x in PossibleTokenMatchReg:
    for y in x:
        z = y.split(":")
        docID = z[0]
        names = z[1].split("/")
        name = names[1]
        name = name.strip()
        if docID not in ambig:
            print("Match for",docID,name,iddict[name]['ID'])
            d[docID]['fullID'] = iddict[name]['ID']
            d[docID]['clearname'] = name
        else: # Really I'd just skip this and commit them to a list/file
            ambigdocs.append(docID)
            ambignames.append(name)
            ambigids.append(iddict[name]['ID'])

-AMBIGUOUS: V18880221Aub LUDVIG [MARIBO BENJAMIN] AUBERT peLMBA
-AMBIGUOUS: V1890HJae HENRIK [BERNHARD] JÆGER peHeJa
Match for V18690926HSTp HEDVIG (ved dåpen Hedevig Cathrine) f. Ibsen STOUSLAND peHSto
Match for V18901219HeG HEDWIG f. Hermann GRØNVOLD peHGr


In [17]:
df = pd.DataFrame.from_dict(d).fillna("N/A").T
df = df.drop(['peID', 'orgID'], axis=1)
df.to_csv("knownDict.csv", sep=',', encoding='utf-8',index=True)

In [35]:
# What's going on with the ambigs, then?
i=0
while i<len(ambigdocs):
    print(ambigdocs[i],ambignames[i],ambigids[i])
    i+=1

V18880221Aub [FREDRIK LUDVIG] ANDREAS [VIBE] AUBERT peAAu
V18880221Aub LUDVIG [MARIBO BENJAMIN] AUBERT peLMBA
V1890HJae HANS [HENRIK] JÆGER peHJa
V1890HJae HENRIK [BERNHARD] JÆGER peHeJa
